I copied the cell below from the GitHub for FunSearch that is available here: https://github.com/kitft/funsearch

If you go to the folder Examples there is a "test_no3points_in_line.py".  You can also go to the folder Playgrounds and see "test_no3points_in_line.ipynb"

I commented out all the stuff where it makes calls to the LLM, the actual "FunSearch" material.  This is still useful for doing trials of building no-three-in-line sets using a random greedy algorithm.  It is also possible to modify the priority function however you want to test greedy constructions that select points with some different (non-random) priority.

In [3]:
# """Finds subsets of an nxn grid that contains 3 points in a line

# On every iteration, improve priority_v1 over the priority_vX methods from previous iterations.
# Make only small changes.
# Try to make the code short.
#"""
import itertools
import numpy as np
#import funsearch
import itertools


def are_collinear(p1, p2, p3):
  """Returns True if the three points are collinear."""
  x1, y1 = p1
  x2, y2 = p2
  x3, y3 = p3
  return (y1 - y2) * (x1 - x3) == (y1 - y3) * (x1 - x2)


#@funsearch.run
def evaluate(n: int) -> int:
  """Returns the size of a point set on the nxn grid that does not contain 3 points in a line."""
  grid_set = solve(n)
  return len(grid_set)

#original version from open source, built by adding lines to empty grid
def solve(n: int) -> np.ndarray:
  """Returns a large non-3-points-in-line subset on the nxn grid."""
  all_points = np.array(list(itertools.product(np.arange(n), repeat=2)), dtype=np.int32)
  #all_integers = np.arange(n)

  # Randomly shuffle the points (and their priorities)
  shuffle_indices = np.random.permutation(len(all_points))
  all_points = all_points[shuffle_indices]

  # Precompute all priorities.
  priorities = np.array([priority(tuple(point), n) for point in all_points])


  # Build `grid_set` greedily, using priorities for prioritization.
  grid_set = np.empty(shape=(0,2), dtype=np.int32)
  while np.any(priorities != -np.inf):
    # Add a vector with maximum priority to `grid_set`, and set priorities of
    # invalidated vectors to `-inf`, so that they never get selected.
    max_index = np.argmax(priorities)
    new_points = all_points[None, max_index]  # [1, n]
    new_point = new_points[0]
    priorities[max_index] = -np.inf

    # Block those points in all_points which lie on a line spanned by a point in grid_set and new_point

    for index in range(len(all_points)):
      if grid_set.size == 0:
          continue
      elif any(are_collinear(new_point, all_points[index], cap) for cap in grid_set):
        priorities[index] = -np.inf

    grid_set = np.concatenate([grid_set, new_point.reshape(1, -1)], axis=0)
    

  return grid_set

#@funsearch.evolve
def priority(el: tuple[int, int], n: int) -> float:
  """Returns the priority with which we want to add `element` to the cap set.  """
  return 0.0

# #alternative version: start with full grid and takeaway
# def solve(n: int) -> np.ndarray:
#     all_points = np.array(list(itertools.product(np.arange(n), repeat=2)), dtype=np.int32)
#     priorities = np.array([priority(tuple(point), n) for point in all_points])
    
#     # Sort points by priority (ascending - remove lowest priority first)
#     indices = np.argsort(priorities)
    
#     grid_set = all_points.copy()
#     for idx in indices:
#         point = all_points[idx]
#         # Try removing this point
#         temp_set = np.delete(grid_set, np.where((grid_set == point).all(axis=1))[0], axis=0)
#         # Check if removing creates a no-three-in-line set
#         if not any(are_collinear(p1, p2, p3) for p1, p2, p3 in itertools.combinations(temp_set, 3)):
#             grid_set = temp_set
#             break
    
#     return grid_set

# def priority(el: tuple[int, int], n: int) -> float:
#     x, y = el
#     # Base priority 
#     base = 1.0 if (x % 3 == 0 or y % 3 == 0) else 0.5
#     # Add distance from center as a factor
#     center_dist = ((x - n/2)**2 + (y - n/2)**2)**0.5 / n
#     return base * (0.5 + center_dist)


# @funsearch.evolve
# def priority(el: tuple[int, int], n: int) -> float:
#   """Returns the priority with which we want to add `element` to the cap set."""
#   return 0.0
     
# """Finds subsets of an nxn grid that contains 3 points in a line

# On every iteration, improve priority_v1 over the priority_vX methods from previous iterations.
# Make only small changes.
# Try to make the code short.
# """
# import itertools
# import numpy as np
# import funsearch

# def are_collinear(p1, p2, p3):
#   """Returns True if the three points are collinear."""
#   x1, y1 = p1
#   x2, y2 = p2
#   x3, y3 = p3
#   return (y1 - y2) * (x1 - x3) == (y1 - y3) * (x1 - x2)


# @funsearch.run
# def evaluate(n: int) -> int:
#   """Returns the size of a point set on the nxn grid that does not contain 3 points in a line."""
#   grid_set = solve(n)
#   return len(grid_set)


# def solve(n: int) -> np.ndarray:
#   """Returns a large non-3-points-in-line subset on the nxn grid."""
#   all_points = np.array(list(itertools.product(np.arange(n), repeat=2)), dtype=np.int32)
  
#   # FIRST PASS
#   # Precompute all priorities.
#   priorities = np.array([priority(tuple(point), n) for point in all_points])

#   # Build `grid_set` greedily, using priorities for prioritization.
#   grid_set = np.empty(shape=(0,2), dtype=np.int32)
#   while np.any(priorities != -np.inf):
#     # Add a vector with maximum priority to `grid_set`, and set priorities of
#     # invalidated vectors to `-inf`, so that they never get selected.
#     max_index = np.argmax(priorities)
#     new_point = all_points[max_index]
#     priorities[max_index] = -np.inf

#     # Block those points in all_points which lie on a line spanned by a point in grid_set and new_point
#     for index in range(len(all_points)):
#       if all_points[index][0] == new_point[0] and all_points[index][1] == new_point[1]:
#         continue  # Skip the new point itself
#       if grid_set.size == 0:
#         continue
#       elif any(are_collinear(new_point, all_points[index], cap) for cap in grid_set):
#         priorities[index] = -np.inf

#     grid_set = np.concatenate([grid_set, new_point.reshape(1, -1)], axis=0)
    
#   return grid_set

# @funsearch.evolve
# def priority(el: tuple[int, int], n: int) -> float:
#   """Returns the priority with which we want to add `element` to the cap set.
#   This priority function should consider both the properties of the point itself
#   and how it relates to an optimal solution strategy.
  
#   A good priority function might:
#   1. Consider the position of the point in relation to the grid boundaries
#   2. Evaluate patterns like the modulo-3 property of coordinates
#   3. Balance the distribution of points across the grid
#   4. Avoid favoring points that tend to create collinearity
#   """
#   return 0.0

In [7]:

len(solve(3))

5

I did 20 trials on each grid size from 10 up to 49.  All I recorded was the average size of the set that was created.

In [ ]:
for n in range(10,50):
    tot = 0
    for i in range(20):

        tot = tot+ len(solve(n))
    print(n,"\t",tot/20,"\n")

10 	 15.85 

11 	 16.5 

12 	 18.85 

13 	 20.25 

14 	 21.75 

15 	 23.25 

16 	 24.4 

17 	 25.65 

18 	 27.4 

19 	 29.0 

20 	 30.35 

21 	 31.8 

22 	 33.5 

23 	 34.95 

24 	 36.3 

25 	 37.8 

26 	 39.0 

27 	 40.95 

28 	 42.1 

29 	 43.75 

30 	 45.15 

31 	 46.7 

32 	 47.45 

33 	 49.85 

34 	 50.95 

35 	 51.7 

36 	 54.0 

37 	 54.55 

38 	 56.3 

39 	 58.15 

40 	 59.25 

41 	 61.55 

42 	 62.75 

43 	 64.0 

44 	 65.15 

45 	 66.55 

46 	 67.9 

47 	 68.9 

48 	 70.6 

49 	 72.65 



I did 20 trials for n = 100.  All I kept was the average size of the set that was created.

In [ ]:
n = 100
tot = 0
for i in range(20):

    tot = tot+ len(solve(n))
print(n,"\t",tot/20,"\n")

100 	 145.25 



I did 20 trials for n = 150, n = 200, and then a smaller number for n = 250.  This time I kept the list of sizes rather than just the average.

In [ ]:
n = 150
tot = 0
for i in range(20):
    v = solve(n)
    print(len(v))
    tot = tot+ len(v)
print("\n",n,"\t",tot/20,"\n")

215
212
219
217
219
220
216
215
215
214
216
216
222
216
215
218
213
214
219
218

 150 	 216.45 



In [ ]:
n = 200
tot = 0
for i in range(20):
    v = solve(n)
    print(len(v))
    tot = tot+ len(v)
print("\n",n,"\t",tot/20,"\n")

283
289
292
283
283
283
286
288
290
282
289
289
292
284
292
288
291
287
285
280

 200 	 286.8 



In [ ]:
n = 250
tot = 0
for i in range(20):
    v = solve(n)
    print(len(v))
    tot = tot+ len(v)
print("\n",n,"\t",tot/20,"\n")

360
358
361
356
358
360
361
354
353
